In [1]:
import pandas as pd
import numpy as np
import random
from scipy.stats import norm, poisson
import statsmodels.api as sm
# from statsmodels.formula.api import poisson
import pandas as pd
import numpy as np

In [2]:
# Basic Poisson Distribution for Scoring 0, 1 or 2 touchdowns

data = pd.read_csv("data/PlayerStats.csv")

# Select the player of interest (example: wide receiver)
wrSelected = 'Tyreek Hill'  # Change this to the wide receiver of interest

# Filter the data to get only the rows for the selected player
wrRows = data.loc[data["player_display_name"] == wrSelected]

# Calculate the average touchdowns per game (mean of the Poisson distribution)
wrAvgTDs = np.average(wrRows['receiving_tds'])

# Calculate the probabilities for scoring 0 or 1 touchdown for the player
touchdown_probs = poisson.pmf(np.arange(3), wrAvgTDs)

# Create a dictionary to display the results
touchdown_probs_dict = {
    f"Touchdowns = {i}": prob for i, prob in enumerate(touchdown_probs)
}

print(touchdown_probs_dict)

{'Touchdowns = 0': 0.5543671658298339, 'Touchdowns = 1': 0.3270367453096861, 'Touchdowns = 2': 0.09646407595465564}


In [3]:
# Adding to Poisson Distribution for Scoring 0, 1 or 2 touchdowns

data = pd.read_csv("data/PlayerStats.csv")

# Select the player of interest (example: wide receiver)
wrSelected = 'Tyreek Hill'  # Change this to the wide receiver of interest

# Filter the data to get only the rows for the selected player
wrRows = data.loc[data["player_display_name"] == wrSelected]

# Select relevant features for the Poisson regression model
X = wrRows[['targets', 'receptions', 'receiving_yards']]  # Add more features as needed
y = wrRows['receiving_tds']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the Poisson regression model
poisson_model = sm.GLM(y, X, family=sm.families.Poisson()).fit()

# Make a copy of the DataFrame to avoid SettingWithCopyWarning
wrRows = wrRows.copy()

# Predict the expected number of touchdowns for each game
# wrRows.loc[:, 'predicted_tds'] = poisson_model.predict(X)
wrRows['predicted_tds'] = poisson_model.predict(X)

# Calculate the probabilities for scoring 0, 1, or 2 touchdowns for each game
touchdown_probs = poisson.pmf(np.arange(3)[:, None], wrRows['predicted_tds'].values)

# Average probabilities across all games
average_probs = touchdown_probs.mean(axis=1)

# Create a dictionary to display the results
touchdown_probs_dict = {
    f"Touchdowns = {i}": prob for i, prob in enumerate(average_probs)
}

# Print the results
print(touchdown_probs_dict)

{'Touchdowns = 0': 0.5959502703390716, 'Touchdowns = 1': 0.27836767713168137, 'Touchdowns = 2': 0.0865499855832598}


In [ ]:
# Basic Monte Carlo for passing yards

qbData = pd.read_csv("data/PlayerStats.csv")

# Select the player you want to simulate
qbSelected = 'Patrick Mahomes'

# Filter the data to get only the rows for the selected player
qbRows = qbData.loc[qbData["player_display_name"] == qbSelected]

# Calculate the average and standard deviation of passing yards per game
qbAvgPassYards = np.average(qbRows['passing_yards'])
qbSTDPassYards = np.std(qbRows['passing_yards'])

# Simulate 10,000 games
simmedNumbers = []
for i in range(0, 10000):
    rP = random.uniform(0, 1)
    passYardsSim = norm.ppf(rP, loc=qbAvgPassYards, scale=qbSTDPassYards)
    simmedNumbers.append(passYardsSim)

# Calculate the average simulated passing yards
avgSimmedGames = np.average(simmedNumbers)

# Print simulated passing yards
print(str(avgSimmedGames))